System Prompt 

You are EcoGranite, an IBM Granite–powered driving efficiency coach. 
Your job is to analyze OBD-II trip summaries and give clear, safe, non-judgmental guidance 
to help a driver reduce fuel consumption, improve smoothness, and prevent engine wear.

Follow these rules:
- Be encouraging and practical.
- Avoid blaming language.
- Give 3–5 specific suggestions.
- Use short paragraphs or bullet points.

User prompt

Here is a driving trip summary:

- Average speed: {avg_speed_kmh} km/h
- Average RPM: {avg_rpm}
- Average fuel consumption: {avg_fuel_l100km} L/100km
- High RPM at low speed events: {high_rpm_low_speed_pct}%
- Long idling: {long_idle_pct}%
- Acceleration events: {accel_events}
- Braking events: {brake_events}

Please provide:
1. A 4–6 sentence overview of the user’s driving efficiency.
2. A bullet list of personalized coaching suggestions.
3. A final short “key takeaway”.

Short Feedback prompt
User prompt

Based on this trip summary, give me VERY short coaching feedback (2 lines max):

- Avg speed: {avg_speed_kmh} km/h
- Avg RPM: {avg_rpm}
- Fuel: {avg_fuel_l100km}
- High RPM/Low Speed: {high_rpm_low_speed_pct}%
- Idling: {long_idle_pct}%

Component-Specific Feedback (RPM, Idling, or Braking)
User prompt

Provide targeted coaching on ONLY the following component: {component}

Trip stats:
- High RPM at low speed: {high_rpm_low_speed_pct}%
- Long idling: {long_idle_pct}%
- Accel/Brake cycles: {accel_events}/{brake_events}

Format:
- 2 sentences explaining the behavior
- 2–3 tips to improve

Where {component} ∈ "RPM" | "Idling" | "Acceleration/Braking"


“Explain Like I’m New” Prompt (beginner-friendly)
User prompt

Explain the driver’s efficiency in a simple, friendly way for a beginner.

Trip:
- Avg speed: {avg_speed_kmh}
- RPM: {avg_rpm}
- Fuel: {avg_fuel_l100km}
- Idling: {long_idle_pct}%

Avoid technical jargon. Keep it gentle and supportive.

In [ ]:
#Imports + load trip summaries
import os
import json
import time
import pandas as pd
import requests

# Load trip-level stats you generated earlier
trip_summary = pd.read_csv("../data/trip_summary_stats.csv")

print(trip_summary.shape)
display(trip_summary.head())


(75, 8)


,Trip,Avg_Speed_kmh,Avg_RPM,Avg_Fuel_L/100km,HighRPM_LowSpeed_%,Long_Idle_%,Accel_Events,Brake_Events
0,2017-07-10_Seat_Leon_KA_KA_Stau.csv,14.455887,940.928115,0.472057,0.01,73.67,340,421
1,2017-07-11_Seat_Leon_KA_KA_Stau.csv,23.052541,1037.744883,0.414721,0.01,83.35,421,444
2,2017-07-11_Seat_Leon_KA_S_Normal.csv,68.238275,1583.488220,0.275587,0.01,89.80,1289,1180
3,2017-07-11_Seat_Leon_S_RT_Frei.csv,58.304813,1445.053577,0.268645,0.01,91.47,627,704
4,2017-07-12_Seat_Leon_RT_S_Normal.csv,52.877656,1416.844637,0.260331,0.00,89.32,1169,1172


In [ ]:
#Prompt builder helper
def build_trip_prompt(row):
    return f"""
Here is a driving trip summary:

- Average speed: {row.Avg_Speed_kmh:.1f} km/h
- Average RPM: {row.Avg_RPM:.0f}
- Average fuel consumption: {row['Avg_Fuel_L/100km']:.1f} L/100km
- High RPM at low speed events: {row['HighRPM_LowSpeed_%']}%
- Long idling: {row['Long_Idle_%']}%
- Acceleration events: {row.Accel_Events}
- Braking events: {row.Brake_Events}

Please provide:
1. Overview (4–6 sentences)
2. Bullet-point coaching suggestions
3. Key takeaway (1 line)
"""


In [ ]:
#Get IAM access token
from dotenv import load_dotenv
load_dotenv()

import os
API_KEY = os.getenv("WATSONX_API_KEY")
if not API_KEY:
    raise ValueError("WATSONX_API_KEY not found.")


def get_iam_access_token(api_key: str) -> str:
    iam_url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": api_key
    }

    r = requests.post(iam_url, headers=headers, data=data)
    if r.status_code != 200:
        raise Exception(f"IAM token error {r.status_code}: {r.text}")

    return r.json()["access_token"]

access_token = get_iam_access_token(API_KEY)
print("✅ Got IAM access token")


✅ Got IAM access token


In [ ]:
#Granite chat call function
WATSONX_CHAT_URL = "https://eu-gb.ml.cloud.ibm.com/ml/v1/text/chat?version=2023-05-29"
PROJECT_ID = "b54bb396-d3a5-410a-b972-e6a719ee761a"
MODEL_ID = "ibm/granite-3-8b-instruct"

def call_granite(prompt_text, access_token, temperature=0.2, max_tokens=350):
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    body = {
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are EcoGranite, a fuel-efficiency driving coach. "
                    "Give supportive, practical eco-driving feedback."
                )
            },
            {"role": "user", "content": prompt_text}
        ],
        "project_id": PROJECT_ID,
        "model_id": MODEL_ID,
        "temperature": temperature,
        "max_tokens": max_tokens
    }

    r = requests.post(WATSONX_CHAT_URL, json=body, headers=headers)
    if r.status_code != 200:
        raise Exception(f"Granite error {r.status_code}: {r.text}")

    return r.json()["choices"][0]["message"]["content"]



Index(['Trip', 'Avg_Speed_kmh', 'Avg_RPM', 'Avg_Fuel_L/100km',
       'HighRPM_LowSpeed_%', 'Long_Idle_%', 'Accel_Events', 'Brake_Events'],
      dtype='object')

In [ ]:
#Test on 1–3 trips first
sample_trips = trip_summary.head(3)

for _, row in sample_trips.iterrows():
    prompt = build_trip_prompt(row)
    feedback = call_granite(prompt, access_token)

    print("="*80)
    print("Trip:", row.Trip)
    print(feedback)


Trip: 2017-07-10_Seat_Leon_KA_KA_Stau.csv
1. Overview: Your driving trip summary indicates a commendable effort towards fuel efficiency. The average speed of 14.5 km/h and fuel consumption of 0.5 L/100km are impressive, suggesting a smooth, steady pace. However, there are areas for improvement, particularly in reducing idling time and optimizing acceleration and braking. The high RPM at low speeds and the significant number of acceleration and braking events suggest room for refining your driving habits to further enhance fuel efficiency.

2. Coaching suggestions:
   - Minimize idling: Try to reduce idling time, as it significantly contributes to fuel waste. If you anticipate a wait, it might be more fuel-efficient to turn off the engine.
   - Smooth acceleration: Gradually build up speed instead of rapid acceleration. This not only improves fuel efficiency but also reduces wear on your vehicle.
   - Anticipate traffic: Look ahead to anticipate traffic flow and adjust your speed accord

In [43]:
#Run on all trips + save JSON
results = []

for i, (_, row) in enumerate(trip_summary.iterrows()):
    prompt = build_trip_prompt(row)

    try:
        feedback = call_granite(prompt, access_token)
    except Exception as e:
        feedback = f"ERROR: {e}"

    results.append({
        "Trip": row.Trip,
        "prompt": prompt.strip(),
        "feedback": feedback.strip()
    })

    # small pause to avoid rate limits
    if (i + 1) % 5 == 0:
        time.sleep(1)

len(results)


75

In [44]:
#export
out_path = "../data/granite_trip_feedback.json"

with open(out_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"✅ Saved Granite feedback to {out_path}")


✅ Saved Granite feedback to ../data/granite_trip_feedback.json
